In [1]:
import torch
import torch.nn as nn

In [13]:
weight = torch.empty((7, 8))
nn.init.trunc_normal_(weight, a=-3, b=3)
w = nn.Parameter(weight)
w

Parameter containing:
tensor([[-0.3832,  0.8606, -1.7517,  0.7076,  2.2103, -1.5405,  1.2070,  0.0998],
        [-0.7179,  0.4025, -0.8924,  1.8451, -0.9918,  0.7694,  0.2267,  0.8916],
        [ 0.4341,  0.1305, -0.8007,  0.4984, -0.9012, -1.3600,  0.3625,  0.1098],
        [-0.8205,  0.3829,  0.1975, -0.6843,  1.2443, -0.2354, -0.4630,  0.9522],
        [ 0.4790,  1.1544,  2.1247, -0.2140, -0.7958,  0.9266,  0.2810, -0.9987],
        [-1.8545, -1.1224,  1.1660, -0.3597, -2.3507,  1.0263, -0.0092, -1.2897],
        [ 1.0308, -1.4569,  0.3120, -2.0290,  0.0909, -1.8946,  0.1431, -0.5837]],
       requires_grad=True)

In [15]:
token_ids = torch.LongTensor([1, 4, 5, 0])
w[token_ids]

tensor([[-0.7179,  0.4025, -0.8924,  1.8451, -0.9918,  0.7694,  0.2267,  0.8916],
        [ 0.4790,  1.1544,  2.1247, -0.2140, -0.7958,  0.9266,  0.2810, -0.9987],
        [-1.8545, -1.1224,  1.1660, -0.3597, -2.3507,  1.0263, -0.0092, -1.2897],
        [-0.3832,  0.8606, -1.7517,  0.7076,  2.2103, -1.5405,  1.2070,  0.0998]],
       grad_fn=<IndexBackward0>)

In [22]:
w.max(dim=-1, keepdim=True).values

tensor([[2.2103],
        [1.8451],
        [0.4984],
        [1.2443],
        [2.1247],
        [1.1660],
        [1.0308]], grad_fn=<MaxBackward0>)

In [35]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
d_k = 6
max_seq_len = 10
theta = 0.5
kd = torch.arange(0, d_k, 2, device=device) / d_k
print(kd)
kd.shape

tensor([0.0000, 0.3333, 0.6667], device='mps:0')


torch.Size([3])

In [36]:
positions = torch.arange(max_seq_len, device=device).unsqueeze(1)
print(positions)
positions.shape

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7],
        [8],
        [9]], device='mps:0')


torch.Size([10, 1])

In [37]:
angles = positions / (theta ** kd)
print(angles)
angles.shape

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.2599,  1.5874],
        [ 2.0000,  2.5198,  3.1748],
        [ 3.0000,  3.7798,  4.7622],
        [ 4.0000,  5.0397,  6.3496],
        [ 5.0000,  6.2996,  7.9370],
        [ 6.0000,  7.5595,  9.5244],
        [ 7.0000,  8.8194, 11.1118],
        [ 8.0000, 10.0794, 12.6992],
        [ 9.0000, 11.3393, 14.2866]], device='mps:0')


torch.Size([10, 3])

In [49]:
d_model = 20
num_heads = 5
qkv_proj = nn.Linear(d_model, d_model * 3)
x = torch.randn((10, 10, d_model))
qkv = qkv_proj(x)
qkv.size()

torch.Size([10, 10, 60])

In [50]:
q, k, v = qkv.split(d_model, dim=2)
assert q.size() == k.size() == v.size()
q.size()

torch.Size([10, 10, 20])

In [48]:
from einops import rearrange
q = rearrange(q, "batch seq_len (h d_k) -> batch h seq_len d_k", h=num_heads)
q.shape

torch.Size([10, 5, 10, 4])